<a href="https://colab.research.google.com/github/takiyama0617/technical_analysis/blob/master/%E6%9C%89%E5%A0%B1%E3%82%AD%E3%83%A3%E3%83%83%E3%83%81%E3%83%A3%E3%83%BC%E3%81%8B%E3%82%89%E8%B2%A1%E5%8B%99%E6%83%85%E5%A0%B1%E3%82%92%E5%8F%96%E5%BE%97%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from google.colab import drive
drive.mount('/content/drive')
BASE_DIR = f'/content/drive/My Drive/workspace/technical_analysis/data/'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1.有報キャッチャーからURLを取得
XBRLのURLリスト dat_download{ str_period }.csv が吐き出される

In [0]:
import os
import csv
import time
import re
import requests
import xml.etree.ElementTree as ET
from tqdm import tqdm
from datetime import datetime, timedelta


class yuho_catcher():
    def __init__(self, since, until, base_dir=None):
        self.csv_tag = ['id', 'title', 'cd', 'url', 'update']
        self.encode_type = 'utf-8'
        self.wait_time = 2  # 間隔が短いと制限がかかる
        self.base_url = 'http://resource.ufocatch.com/atom/edinetx/'
        self.namespace = '{http://www.w3.org/2005/Atom}'
        self.out_of_since = False
        self.since = since
        self.until = until
        self.file_info_str = since.strftime(
            '_%y%m%d_') + until.strftime('%y%m%d')
        self.base_path = f'{ os.getcwd() if base_dir==None else base_dir }'

    def get_link_info_str(self, ticker_symbol):
        url = self.base_url + str(ticker_symbol)
        count, retry = 0, 3
        while True:
            try:
                response = requests.get(url)
                return response.text
            except Exception:
                print(f'{ticker_symbol} のアクセスに失敗しました。[ {count} ]')
                if count < retry:
                    count += 1
                    time.sleep(3)
                    continue
                else:
                    raise

    def parse_xml(self, string):
        ET_tree = ET.fromstring(string)
        ET.register_namespace('', self.namespace[1:-1])
        return ET_tree

    def get_link(self, tree):
        yuho_dict = {}
        # xmlのentry毎にfor
        for el in tree.findall('.//'+self.namespace+'entry'):
            title = el.find(self.namespace+'title').text
            if not self.is_yuho(title):
                continue
            updated = el.find(self.namespace+'updated').text
            checked = self.time_check(updated)
            if not checked['until']:
                continue
            if not checked['since']:
                self.out_of_since = True
                return yuho_dict
            _id = el.find(self.namespace+'id').text
            links = el.findall('./'+self.namespace+'link[@type="text/xml"]')
            for link in links:
                if '.xbrl' in link.attrib['href'] and 'PublicDoc' in link.attrib['href']:
                    url = link.attrib['href']
                    break
                else:
                    continue
            else:
                continue
            cd = re.sub(r'^【(\w+)】.*', r"\1", title)
            yuho_dict[_id] = {'id': _id, 'title': title,
                              'cd': cd, 'url': url, 'update': updated}
        return yuho_dict

    def is_yuho(self, title):
        if u'有価証券報告書' in str(title) and u'株式会社' in str(title) and u'内国投資信託受益証券' not in str(title):
            return True
        return False

    def time_check(self, update):
        updated_time = datetime.strptime(update, '%Y-%m-%dT%H:%M:%S+09:00')
        return {'since': updated_time >= self.since, 'until': updated_time < self.until}

    def dump_file(self, file, info_dict, tag, encode_type):
        with open(os.path.join(self.base_path, file), 'w', encoding=encode_type) as of:
            writer = csv.DictWriter(of, tag, lineterminator='\n')
            writer.writeheader()
            for key in info_dict:
                writer.writerow(info_dict[key])

    def craete_xbrl_url_json_each_symbols(self, list_symbols):
        print(
            f'since:{ self.since } ,until:{ self.until } ({ self.file_info_str })')
        i, result_dict = 0, {}
        for t_symbol in tqdm(list_symbols):
            response_string = self.get_link_info_str(f'query/{t_symbol}')
            ET_tree = self.parse_xml(response_string)
            info_dict = self.get_link(ET_tree)
            if len(info_dict) > 0:
                for key in info_dict:
                    result_dict[key] = info_dict[key]
                self.dump_file(
                    f'dat_download{ self.file_info_str }.csv', result_dict, self.csv_tag, self.encode_type)
            time.sleep(self.wait_time)
            i += 1
        print('complete a download!!')

    def craete_xbrl_url_json_from_latest(self):
        print(
            f'since:{ self.since } ,until:{ self.until } ({ self.file_info_str })')
        i, result_dict = 0, {}
        while True:
            page = 1 + i
            print(f'page{page}, loading...')
            response_string = self.get_link_info_str(page)
            ET_tree = self.parse_xml(response_string)
            info_dict = self.get_link(ET_tree)
            if len(info_dict) > 0:
                for key in info_dict:
                    result_dict[key] = info_dict[key]
                self.dump_file(
                    f'dat_download{ self.file_info_str }.csv', result_dict, self.csv_tag, self.encode_type)
            time.sleep(self.wait_time)
            if self.out_of_since:
                break
            i += 1
        print('complete a download!!')


In [21]:
from datetime import datetime

since = datetime.strptime('2020-04-01' ,'%Y-%m-%d')
until = datetime.strptime('2020-05-03' ,'%Y-%m-%d')
yuho = yuho_catcher( since ,until, base_dir=BASE_DIR)
yuho.craete_xbrl_url_json_from_latest()
# yuho.craete_xbrl_url_json_each_symbols( [7203] )

since:2020-04-01 00:00:00 ,until:2020-05-03 00:00:00 (_200401_200503)
page1, loading...
page2, loading...
page3, loading...
page4, loading...
page5, loading...
page6, loading...
page7, loading...
page8, loading...
page9, loading...
page10, loading...
page11, loading...
complete a download!!


# 2.XBRLのURLリストからファイルをダウンロードし、CSV変換

dat_download_{from}_{to}.csvからXBRLファイルをダウンロード

ダウンロードしたXBRLファイルから必要情報を抜き出し、eggs_{from}_{to}.csvを作成する



In [22]:
!pip install python-xbrl

In [0]:
import os
import re
import csv
import time
import urllib
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from xbrl import XBRLParser

default_tag = ['file_nm', 'element_id', 'amount']
custom_tag = ['unit_ref', 'decimals', 'contextref']
encode_type = 'utf-8'


class XbrlParser(XBRLParser):
    def __init__(self, info_dict):
        self.info_dict = info_dict
        self.xbrl_url = info_dict['url']
        xbrl_filename = self.xbrl_url.split('/')[-1]
        directory_path = f'{ os.getcwd() }/xbrl_files'
        self.__make_directory(directory_path)
        self.xbrl_filepath = f'{ directory_path }/{ xbrl_filename }.xbrl'

    def __make_directory(self, dir_name):
        if not os.path.exists(dir_name):
            os.mkdir(dir_name)

    def __download_xbrl_file(self):
        counter, retry = 0, 3
        while True:
            try:
                urllib.request.urlretrieve(self.xbrl_url, self.xbrl_filepath)
                return True
            except Exception as e:
                if counter < retry:
                    print(f'XBRLファイルのDLに失敗しました（{ counter }）{ self.xbrl_url }')
                    time.sleep(3)
                    counter += 1
                else:
                    print('上手く行かないので次の処理に移ります')
                    return False

    def parse_xbrl(self):
        DL_is_succeeded = self.__download_xbrl_file()
        if DL_is_succeeded:
            with open(self.xbrl_filepath, 'r', encoding='utf-8') as of:
                xbrl = XBRLParser.parse(of)  # beautiful soup type object
        else:
            return
        result_dicts = {}
        _idx = 0
        name_space = 'jp*'
        for node in xbrl.find_all(name=re.compile(name_space+':*')):
            if self.ignore_pattern(node):
                continue
            row_dict = {}
            row_dict['file_nm'] = self.xbrl_filepath.rsplit(os.sep, 1)[1]
            row_dict['element_id'] = node.name
            row_dict['amount'] = node.string
            for tag in custom_tag:
                row_dict[tag] = self.get_attrib_value(node, tag)
            result_dicts[_idx] = row_dict
            _idx += 1
        return result_dicts

    def ignore_pattern(self, node):
        if 'xsi:nil' in node.attrs:
            if node.attrs['xsi:nil'] == 'true':
                return True
        if not isinstance(node.string, str):
            return True  # 結果が空の場合は対象外にする
        if str(node.string).find(u'\n') > -1:
            return True  # 改行コードが結果にある場合対象外にする
        if u'textblock' in str(node.name):
            return True  # text文字列は解析に利用できないため、対象外
        return False

    def get_attrib_value(self, node, attrib):
        if attrib in node.attrs.keys():
            return node.attrs[attrib]
        else:
            return None


class parse_operator():
    def __init__(self, df, str_period, base_dir):
        self.df = df
        self.str_period = str_period
        self.base_path = base_dir

    def __dump_file(self, writer, info_dicts):
        _idx = 0
        while _idx < len(info_dicts):
            row_dict = info_dicts[_idx]
            writer.writerow(row_dict)
            _idx += 1

    def xbrl_to_csv(self):
        file = f'eggs_{ self.str_period }.csv'
        with open(os.path.join(self.base_path, file), 'w', encoding=encode_type) as of:
            resultCsvWriter = csv.DictWriter(
                of, default_tag + custom_tag, lineterminator='\n')
            resultCsvWriter.writeheader()
            for i in tqdm(range(len(self.df.index))):
                row = self.df.iloc[i]
                xp = XbrlParser(row.to_dict())
                info_dicts = xp.parse_xbrl()
                self.__dump_file(resultCsvWriter, info_dicts)
        print('completed conversions!!')


def xbrl_operator(start, end, loop_month=1, restart=0, base_dir=None):
    big_period = start.strftime('%y%m%d_')+end.strftime('%y%m%d')
    base_path = f'{ os.getcwd() if base_dir==None else base_dir }'
    file = f'dat_download_{ big_period }.csv'
    info_df = pd.read_csv(os.path.join(base_path, file),
                          parse_dates=['update'])
    since = start + relativedelta(months=(restart))
    until = start + relativedelta(months=(loop_month + restart))
    until = until if until < end else end

    while True:
        target_df = info_df.loc[(info_df['update'] >= since) & (
            info_df['update'] < until)]
        small_period = since.strftime('%y%m%d_')+until.strftime('%y%m%d')
        print(f'since:{ since } ,until:{ until } ({ small_period })')
        xbrl_parse_operator = parse_operator(
            target_df, small_period, base_path)
        xbrl_parse_operator.xbrl_to_csv()
        since = until
        if since >= end:
            break
        until = until + relativedelta(months=loop_month)
        until = until if until < end else end


In [25]:
from datetime import datetime

start = datetime.strptime('2020-04-01 +0900', '%Y-%m-%d %z')
end = datetime.strptime('2020-05-03 +0900', '%Y-%m-%d %z')
xbrl_operator(start, end, 3, base_dir=BASE_DIR)


  0%|          | 0/124 [00:00<?, ?it/s]

since:2020-04-01 00:00:00+09:00 ,until:2020-05-03 00:00:00+09:00 (200401_200503)


100%|██████████| 124/124 [09:27<00:00,  4.58s/it]

completed conversions!!
